In [18]:
import pandas as pd
import requests

In [19]:
from retrieve_post_by_userid import RetrievePostByUserId
r = RetrievePostByUserId(sessionid="")

In [20]:

import re
def get_threads_userpk(username:str):
    
    headers = {
        "Accept": "*/*",
        "Accept-Language": "it-IT,it;q=0.8",
        "Content-Type": "application/x-www-form-urlencoded",
        "Origin": "https://www.threads.net",
        "Priority": "u=1, i",
        "Referer": f"https://www.threads.net/@{username}",
        "Sec-CH-UA": '"Not(A:Brand";v="99", "Brave";v="133", "Chromium";v="133"',
        "Sec-CH-UA-Full-Version-List": '"Not(A:Brand";v="99.0.0.0", "Brave";v="133.0.0.0", "Chromium";v="133.0.0.0"',
        "Sec-CH-UA-Mobile": "?0",
        "Sec-CH-UA-Model": "",
        "Sec-CH-UA-Platform": '"Windows"',
        "Sec-CH-UA-Platform-Version": '"19.0.0"',
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Sec-GPC": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
        "X-ASBD-ID": "129477",
        "X-FB-LSD": "AVrgJ17PSUU"
    }
    # cookies = {
    #         "sessionid": sessionid
    # }
    data = {
        "client_previous_actor_id": "",
        "route_url": f"/@{username}",
        "routing_namespace": "barcelona_web",
        "__user": "0",
        "__a": "1",
        "__req": "m",
        "__hs": "20128.HYP:barcelona_web_pkg.2.1...0",
        "dpr": "1",
        "__comet_req": "29",
        "lsd": "AVrgJ17PSUU",
    }
    response = requests.post(
            url = "https://www.threads.net/ajax/navigation/",
            headers=headers,
            data = data
                # cookies=cookies
            )
    try:
        if "viewer does not follow private user" in response.text:
            return None
        text = response.text
        user_id_key_value = re.search('"user_id":"(\\d+)",', text).group()
        user_id = re.search('\\d+', user_id_key_value).group()
    except:
        raise Exception()
    return user_id

id = get_threads_userpk("mlssfshn")
print(id)

187264532


In [11]:
df = pd.read_csv("followersAI.csv")
df

,thread_user_pk,thread_follower_pk,thread_username,thread_follower_username
0,6.331078e+10,6.408954e+10,mlssfshn,gabriel_lagon
1,6.338794e+10,6.965537e+10,thisisaltgian,beiyo_
2,6.338794e+10,6.313327e+10,thisisaltgian,realwanita
3,6.338794e+10,6.322924e+10,thisisaltgian,pelumi__i
4,6.338794e+10,6.342134e+10,thisisaltgian,olufisola_
...,...,...,...,...
25182,6.329816e+10,6.322772e+10,msmarshellw,corinnespartana
25183,6.798282e+10,6.322772e+10,affyayomikun,corinnespartana
25184,6.825805e+10,6.322772e+10,quentin.d.h,corinnespartana
25185,6.325159e+10,6.322772e+10,ohnelleyeah,corinnespartana


In [12]:
def analize_post(data_dict,thread_user_pk):
    extracted_posts = []
    for edge in data_dict.get("data", {}).get("mediaData", {}).get("edges", []):
        node = edge.get("node", {})
        for item in node.get("thread_items", []):
            post = item.get("post", {})
            extracted = {
                "id": post.get("id"),
                "post_pk": post.get("pk"),
                "caption_text": post.get("caption", {}).get("text") if post.get("caption") else None,
                "like_count": post.get("like_count"),
                "taken_at": post.get("taken_at"),
                "username": post.get("user", {}).get("username"),
                "user_pk": post.get("user", {}).get("pk"),
                "quote_count": post.get("text_post_app_info", {}).get("quote_count"),
                "repost_count": post.get("text_post_app_info", {}).get("repost_count"),
                "reshare_count": post.get("text_post_app_info", {}).get("reshare_count"),
                "thread_user_pk": thread_user_pk,
            }
            extracted_posts.append(extracted)
            
    return extracted_posts


In [36]:
dfpost = pd.read_csv("postAI.csv")
dfpost
user_processed = dfpost["thread_user_pk"].astype(str).unique()
len(user_processed)

3077

In [37]:
user_processed =list(user_processed)
user_processed.append("69153761497.0")
user_processed.append("63069579229.0")
user_processed.append("64147865290.0")

In [38]:
previus = ""
for idx ,row in df.iterrows():
    if str(row["thread_follower_username"]) == previus:
        continue
    if not str(row["thread_follower_pk"]) in user_processed:
        previus = row["thread_follower_username"]
        print(row["thread_follower_pk"], " ",row["thread_follower_username"])
        try:
            id = get_threads_userpk(row["thread_follower_username"])
        except:
            if "46962366031" == str(get_threads_userpk("shawnecesellsatl")):
                user_processed.append(row["thread_follower_pk"])
                continue
        posts = r.retrieve_thread_by_userid(userid=str(id))

        if not id or not posts["data"]["mediaData"]["edges"]:
            temp_df = pd.DataFrame([{
                "id": None,
                "post_pk": None,
                "caption_text": None,
                "like_count": None,
                "taken_at": None,
                "username": row["thread_follower_username"],
                "user_pk": id if id else "",
                "thread_follower_pk": row["thread_follower_pk"],
                "quote_count": None,
                "repost_count": None,
                "reshare_count": None
            }])
        else:
            temp_df = pd.DataFrame(analize_post(posts,row["thread_follower_pk"]))
        user_processed.append(str(row["thread_follower_pk"]))
        temp_df.to_csv("postAI.csv", mode='a', header=False, index=False)
        print("Processed ", idx, "on ", len(df))

69655371289.0   beiyo_
Processed  1 on  25187
68027286491.0   mercyblackstarr
Processed  9 on  25187
69073676544.0   smart_coded_
Processed  11 on  25187
70271496759.0   _h.a.r.m.o.n.y___
Processed  16 on  25187
63078031507.0   x.akintola
Processed  17 on  25187
63384758057.0   _izzy_boy__
Processed  27 on  25187
63063133468.0   o_oreoluwaa
Processed  48 on  25187
63501488721.0   louisngatale
Processed  56 on  25187
63248745746.0   dansteveade
Processed  60 on  25187
70506914416.0   folorunso_xx
Processed  62 on  25187
63061636525.0   glow_rified_
Processed  66 on  25187
63427812058.0   iam_ajibs
Processed  73 on  25187
63445161605.0   mayomi_tee
Processed  76 on  25187
63098729171.0   aniebietoluwa
Processed  79 on  25187
69609903410.0   focusgreene
Processed  83 on  25187
71454521257.0   johnny_debrah
Processed  89 on  25187
63386772500.0   moabiodun
Processed  94 on  25187
62859951938.0   godspraise40
Processed  97 on  25187
67288943419.0   gbenga_ogunsanwo
Processed  104 on  25187


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [34]:
dfpost

,id,post_pk,caption_text,like_count,taken_at,username,user_pk,quote_count,repost_count,reshare_count,thread_user_pk
0,3558787428782179764_187264532,3.558787e+18,Behind the scenes,0.0,1.738461e+09,mlssfshn,187264532,0.0,0.0,NaN,6.331078e+10
1,3557970465522674878_187264532,3.557970e+18,A little over 24 hours left to vote.\n\nhttps:...,0.0,1.738363e+09,mlssfshn,187264532,0.0,0.0,NaN,6.331078e+10
2,3557782854629238181_187264532,3.557783e+18,"Galaxy Quest \nLots of cutting holes, playing ...",1.0,1.738341e+09,mlssfshn,187264532,0.0,0.0,NaN,6.331078e+10
3,3557776470965660229_187264532,3.557776e+18,Black Hole Sun\nOver 5 pounds of cornhusk in m...,0.0,1.738340e+09,mlssfshn,187264532,0.0,0.0,NaN,6.331078e+10
4,3556000079513982405_187264532,3.556000e+18,Over due to post videos of my work for @salina...,0.0,1.738128e+09,mlssfshn,187264532,0.0,0.0,NaN,6.331078e+10
...,...,...,...,...,...,...,...,...,...,...,...
71682,3558300046839608774_44105325741,3.558300e+18,Sono convinto che gli psicofarmaci (questo è u...,0.0,1.738402e+09,sigfridoscivelli,44105325741,0.0,0.0,NaN,6.879100e+10
71683,3558288587028027828_44105325741,3.558289e+18,"Se c'è un bambino bullizzato, si chiude l'isti...",16.0,1.738401e+09,sigfridoscivelli,44105325741,0.0,0.0,NaN,6.879100e+10
71684,3558285662104847638_44105325741,3.558286e+18,"Le elementari e le medie ""aspirano"" la voglia ...",1.0,1.738401e+09,sigfridoscivelli,44105325741,0.0,0.0,NaN,6.879100e+10
71685,3558004634685589276_44105325741,3.558005e+18,"Studiate se siete dei genî, altrimenti andate ...",3.0,1.738367e+09,sigfridoscivelli,44105325741,0.0,0.0,NaN,6.879100e+10
